In [ ]:
import pandas as pd
import numpy as np
import itertools
import datetime
import matplotlib.pyplot as plt
from datetime import *
from datetime import datetime, timedelta, date
%load_ext google.colab.data_table

# Standard plotly imports
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [ ]:
#Change the date to the previous day
query = """
SELECT * FROM `i-dss-sports-data.rd_temp.video_dash_main` WHERE platform='Total' and day<CURRENT_DATE() order by Day
"""
project_id = "i-dss-sports-data"
df_bq = pd.read_gbq(query=query, project_id=project_id, dialect='standard')

In [ ]:
df_bq.head()

,window_start,window_end,Day,Time_Window,Hour,platform,stream,Unique_Devices,Hours_Total,Minutes_Total,Seconds_Total,AMA,Minutes_Avg,Minutes_Median,Sessions_R,New_sessions_R,Existing_sessions_R,Session_Ends,New_Segment_Completes,Existing_Segment_Completes,Bounce_Percentage_1,Active_Percentage_1,Engaged_Percentage_1,Passive_Percentage_1,Bounce_Percentage_2,Active_Percentage_2,Engaged_Percentage_2,Passive_Percentage_2,Unique_Devices_Day,Sessions_Day,Minutes_Day,AMA_Day,Minutes_Avg_Day,Minutes_Median_Day,ad_minutes,streams,ad_starts
0,2021-08-18 09:00:00,2021-08-18 10:00:00,2021-08-18,09:00,09,Total,HQ Live,6219,1420.0,85201.4,5112082,1420.0,13.08,1.00,6520,4893,1627,4886,10.68,73.45,40.1,31.0,25.6,3.3,0.3,3.2,42.0,54.4,64537,74197,1976125.6,1372.3,26.63,0.55,357312.8,72623,837035
1,2021-08-18 14:00:00,2021-08-18 15:00:00,2021-08-18,14:00,02,Total,HQ Live,9572,1884.6,113073.1,6784384,1884.6,11.46,0.87,9878,7496,2382,7559,9.58,73.62,37.3,48.3,12.7,1.6,0.8,2.6,37.5,59.0,64537,74197,1976125.6,1372.3,26.63,0.55,357312.8,72623,837035
2,2021-08-18 18:00:00,2021-08-18 19:00:00,2021-08-18,18:00,06,Total,HQ Live,6183,1666.7,100001.3,6000077,1666.7,15.44,0.75,6501,4399,2102,4077,21.70,77.24,45.6,23.7,27.4,3.4,0.3,1.8,38.3,59.6,64537,74197,1976125.6,1372.3,26.63,0.55,357312.8,72623,837035
3,2021-08-18 08:00:00,2021-08-18 09:00:00,2021-08-18,08:00,08,Total,HQ Live,3672,1155.5,69331.9,4159911,1155.5,17.53,1.03,3962,2534,1428,2364,22.17,75.92,40.9,25.9,26.3,6.9,0.2,1.5,39.3,59.0,64537,74197,1976125.6,1372.3,26.63,0.55,357312.8,72623,837035
4,2021-08-18 11:00:00,2021-08-18 12:00:00,2021-08-18,11:00,11,Total,HQ Live,3490,1542.8,92570.9,5554251,1542.8,24.43,1.63,3796,1949,1847,1774,32.04,77.43,35.9,22.7,29.5,12.0,0.1,0.8,34.8,64.3,64537,74197,1976125.6,1372.3,26.63,0.55,357312.8,72623,837035


In [ ]:
#Read CSV
df_bq = pd.read_csv('..//.csv')
# This example reads the CSV file data.csv, with ; as a separator, the first row as the header, the first column as an index, skipping the first 5 rows and replacing N/A with NaN.
df = pd.read_csv('...//.csv', sep=';', header=0, index_col=0, skiprows=5, na_values='N/A')

In [ ]:
# Describe
df_bq.describe()

,Unique_Devices,Hours_Total,Minutes_Total,Seconds_Total,AMA,Minutes_Avg,Minutes_Median,Sessions_R,New_sessions_R,Existing_sessions_R,Session_Ends,New_Segment_Completes,Existing_Segment_Completes,Bounce_Percentage_1,Active_Percentage_1,Engaged_Percentage_1,Passive_Percentage_1,Bounce_Percentage_2,Active_Percentage_2,Engaged_Percentage_2,Passive_Percentage_2,Unique_Devices_Day,Sessions_Day,Minutes_Day,AMA_Day,Minutes_Avg_Day,Minutes_Median_Day,ad_minutes,streams,ad_starts
count,26448.000000,26448.000000,2.644800e+04,2.644800e+04,26448.000000,26448.000000,26429.000000,26448.000000,26448.000000,26448.000000,26448.000000,26429.000000,23162.000000,26429.000000,26429.000000,26429.000000,26429.000000,26352.000000,25803.000000,26239.000000,26391.000000,2.644800e+04,2.644800e+04,2.644800e+04,26448.000000,26448.000000,26448.000000,2.644800e+04,2.644800e+04,2.644800e+04
mean,8556.946839,2090.682543,1.254410e+05,7.526457e+06,2090.682543,20.281931,9.443569,8820.656836,5935.085715,2885.571121,6016.237863,25.281793,60.697684,38.765352,23.242264,34.531666,3.461644,0.286627,2.159020,51.551744,46.560483,1.221121e+05,1.440331e+05,3.010583e+06,2090.682123,32.320172,8.981143,3.280524e+05,2.059039e+05,8.554765e+05
std,20769.612203,7459.843072,4.475906e+05,2.685544e+07,7459.843072,11.182569,38.165690,21385.099945,14242.232373,9408.059209,14393.520305,17.638317,17.299470,17.636399,10.632328,19.283687,4.770304,1.299283,2.763109,23.183396,24.057379,1.623365e+05,1.981848e+05,5.057385e+06,3512.071811,22.572936,12.577023,5.012642e+05,3.033138e+05,1.376612e+06
min,1.000000,0.000000,1.000000e-01,4.000000e+00,0.000000,0.010000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.710000e+02,6.050000e+02,1.776630e+04,12.300000,2.510000,0.000000,0.000000e+00,7.580000e+02,0.000000e+00
25%,199.000000,87.875000,5.271850e+03,3.163128e+05,87.875000,11.600000,0.520000,201.000000,91.000000,93.000000,95.000000,13.930000,52.380000,27.900000,16.100000,21.700000,1.100000,0.000000,0.600000,37.500000,29.300000,7.257000e+03,7.833000e+03,3.774270e+05,262.100000,14.140000,0.530000,2.484100e+03,1.222000e+04,5.818000e+03
50%,2603.000000,778.550000,4.671290e+04,2.802776e+06,778.550000,17.620000,1.090000,2680.500000,1493.000000,967.000000,1578.500000,20.290000,61.970000,38.500000,23.500000,29.400000,2.400000,0.200000,1.400000,52.800000,44.400000,7.359000e+04,8.352350e+04,1.491977e+06,1036.100000,22.605000,0.860000,1.694028e+05,1.031310e+05,4.125020e+05
75%,7875.000000,1718.325000,1.031012e+05,6.186070e+06,1718.325000,26.820000,3.600000,8097.250000,5635.250000,2323.000000,5757.250000,30.120000,71.110000,50.000000,30.000000,42.800000,4.100000,0.400000,3.000000,66.700000,61.300000,1.569430e+05,1.836590e+05,2.929813e+06,2034.600000,46.970000,15.550000,4.473503e+05,2.402850e+05,1.147825e+06
max,412546.000000,254538.500000,1.527231e+07,9.163387e+08,254538.500000,60.000000,3405.570000,413298.000000,280748.000000,282571.000000,404341.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,99.900000,100.000000,1.018814e+06,1.313520e+06,5.242870e+07,36408.800000,219.520000,69.680000,4.775408e+06,2.004470e+06,1.482598e+07


In [ ]:
df_bq.describe(include='all')  # include all columns
df_bq.describe(exclude='number') # exclude numerical columns

<ipython-input-20-9214ccc9d642>:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  df_bq.describe(include='all')  # include all columns
<ipython-input-20-9214ccc9d642>:2: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  df_bq.describe(exclude='number') # exclude numerical columns


,window_start,window_end,Day,Time_Window,Hour,platform,stream
count,26448,26448,26448,26448,26448,26448,26448
unique,13224,13224,551,24,12,1,2
top,2021-08-18 09:00:00,2021-08-18 10:00:00,2021-08-18,09:00,09,Total,HQ Live
freq,2,2,48,1102,2204,26448,13224
first,2021-08-18 00:00:00,2021-08-18 01:00:00,NaN,NaN,NaN,NaN,NaN
last,2023-02-20 23:00:00,2023-02-21 00:00:00,NaN,NaN,NaN,NaN,NaN


In [ ]:
#Info
df_bq.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26448 entries, 0 to 26447
Data columns (total 37 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   window_start                26448 non-null  datetime64[ns]
 1   window_end                  26448 non-null  datetime64[ns]
 2   Day                         26448 non-null  dbdate        
 3   Time_Window                 26448 non-null  object        
 4   Hour                        26448 non-null  object        
 5   platform                    26448 non-null  object        
 6   stream                      26448 non-null  object        
 7   Unique_Devices              26448 non-null  Int64         
 8   Hours_Total                 26448 non-null  float64       
 9   Minutes_Total               26448 non-null  float64       
 10  Seconds_Total               26448 non-null  Int64         
 11  AMA                         26448 non-null  float64   

In [ ]:
# iloc() select first row
df_bq.iloc[0]

window_start                  2021-08-18 09:00:00
window_end                    2021-08-18 10:00:00
Day                                    2021-08-18
Time_Window                                 09:00
Hour                                           09
platform                                    Total
stream                                    HQ Live
Unique_Devices                               6219
Hours_Total                                1420.0
Minutes_Total                             85201.4
Seconds_Total                             5112082
AMA                                        1420.0
Minutes_Avg                                 13.08
Minutes_Median                                1.0
Sessions_R                                   6520
New_sessions_R                               4893
Existing_sessions_R                          1627
Session_Ends                                 4886
New_Segment_Completes                       10.68
Existing_Segment_Completes                  73.45


In [ ]:
# Select the first two rows
print(df_bq.iloc[:2])
# Select the first column
print(df_bq.iloc[:, 0])
# Select the first two columns
print(df_bq.iloc[:, :2])
# Select the (1, 1) element
print(df_bq.iloc[1, 1])

In [ ]:
#loc()
# Select the column name 'Seconds_Total'
print(df_bq.loc[:,'Seconds_Total'])

0        5112082
1        6784384
2        6000077
3        4159911
4        5554251
          ...   
26443      83945
26444    3093694
26445      31330
26446    1264984
26447       9476
Name: Seconds_Total, Length: 26448, dtype: Int64


In [ ]:
# Select the columns named 'AMA' and 'Hour'
print(df_bq.loc[:, ['AMA', 'Hour']])

          AMA Hour
0      1420.0   09
1      1884.6   02
2      1666.7   06
3      1155.5   08
4      1542.8   11
...       ...  ...
26443    23.3   09
26444   859.4   10
26445     8.7   10
26446   351.4   11
26447     2.6   11

[26448 rows x 2 columns]


In [ ]:
#assign ()
# Pandas’ .assign() function is used to add new columns to a DataFrame, based on the computation of existing columns. It allows you to add new columns to a DataFrame without modifying the original dataframe. The function returns a new DataFrame with the added columns.
df_new = df_bq.assign(ama_plus_5=df_bq['AMA'] + 5)
df_new.head()

,window_start,window_end,Day,Time_Window,Hour,platform,stream,Unique_Devices,Hours_Total,Minutes_Total,Seconds_Total,AMA,Minutes_Avg,Minutes_Median,Sessions_R,New_sessions_R,Existing_sessions_R,Session_Ends,New_Segment_Completes,Existing_Segment_Completes,Bounce_Percentage_1,Active_Percentage_1,Engaged_Percentage_1,Passive_Percentage_1,Bounce_Percentage_2,Active_Percentage_2,Engaged_Percentage_2,Passive_Percentage_2,Unique_Devices_Day,Sessions_Day,Minutes_Day,AMA_Day,Minutes_Avg_Day,Minutes_Median_Day,ad_minutes,streams,ad_starts,ama_plus_5
0,2021-08-18 09:00:00,2021-08-18 10:00:00,2021-08-18,09:00,09,Total,HQ Live,6219,1420.0,85201.4,5112082,1420.0,13.08,1.00,6520,4893,1627,4886,10.68,73.45,40.1,31.0,25.6,3.3,0.3,3.2,42.0,54.4,64537,74197,1976125.6,1372.3,26.63,0.55,357312.8,72623,837035,1425.0
1,2021-08-18 14:00:00,2021-08-18 15:00:00,2021-08-18,14:00,02,Total,HQ Live,9572,1884.6,113073.1,6784384,1884.6,11.46,0.87,9878,7496,2382,7559,9.58,73.62,37.3,48.3,12.7,1.6,0.8,2.6,37.5,59.0,64537,74197,1976125.6,1372.3,26.63,0.55,357312.8,72623,837035,1889.6
2,2021-08-18 18:00:00,2021-08-18 19:00:00,2021-08-18,18:00,06,Total,HQ Live,6183,1666.7,100001.3,6000077,1666.7,15.44,0.75,6501,4399,2102,4077,21.70,77.24,45.6,23.7,27.4,3.4,0.3,1.8,38.3,59.6,64537,74197,1976125.6,1372.3,26.63,0.55,357312.8,72623,837035,1671.7
3,2021-08-18 08:00:00,2021-08-18 09:00:00,2021-08-18,08:00,08,Total,HQ Live,3672,1155.5,69331.9,4159911,1155.5,17.53,1.03,3962,2534,1428,2364,22.17,75.92,40.9,25.9,26.3,6.9,0.2,1.5,39.3,59.0,64537,74197,1976125.6,1372.3,26.63,0.55,357312.8,72623,837035,1160.5
4,2021-08-18 11:00:00,2021-08-18 12:00:00,2021-08-18,11:00,11,Total,HQ Live,3490,1542.8,92570.9,5554251,1542.8,24.43,1.63,3796,1949,1847,1774,32.04,77.43,35.9,22.7,29.5,12.0,0.1,0.8,34.8,64.3,64537,74197,1976125.6,1372.3,26.63,0.55,357312.8,72623,837035,1547.8


In [ ]:
#Query()
# Pandas’ .query() function allows you to filter a DataFrame based on a Boolean expression. It allows you to select rows from a DataFrame using a query string similar to SQL. The function returns a new DataFrame containing only the rows that satisfy the Boolean expression.

# Select rows where AMA is greater than 1000 and Sessions_R is less than 1000
df_query = df_bq.query('AMA > 1000')
df_query.head()

,window_start,window_end,Day,Time_Window,Hour,platform,stream,Unique_Devices,Hours_Total,Minutes_Total,Seconds_Total,AMA,Minutes_Avg,Minutes_Median,Sessions_R,New_sessions_R,Existing_sessions_R,Session_Ends,New_Segment_Completes,Existing_Segment_Completes,Bounce_Percentage_1,Active_Percentage_1,Engaged_Percentage_1,Passive_Percentage_1,Bounce_Percentage_2,Active_Percentage_2,Engaged_Percentage_2,Passive_Percentage_2,Unique_Devices_Day,Sessions_Day,Minutes_Day,AMA_Day,Minutes_Avg_Day,Minutes_Median_Day,ad_minutes,streams,ad_starts
0,2021-08-18 09:00:00,2021-08-18 10:00:00,2021-08-18,09:00,09,Total,HQ Live,6219,1420.0,85201.4,5112082,1420.0,13.08,1.00,6520,4893,1627,4886,10.68,73.45,40.1,31.0,25.6,3.3,0.3,3.2,42.0,54.4,64537,74197,1976125.6,1372.3,26.63,0.55,357312.8,72623,837035
1,2021-08-18 14:00:00,2021-08-18 15:00:00,2021-08-18,14:00,02,Total,HQ Live,9572,1884.6,113073.1,6784384,1884.6,11.46,0.87,9878,7496,2382,7559,9.58,73.62,37.3,48.3,12.7,1.6,0.8,2.6,37.5,59.0,64537,74197,1976125.6,1372.3,26.63,0.55,357312.8,72623,837035
2,2021-08-18 18:00:00,2021-08-18 19:00:00,2021-08-18,18:00,06,Total,HQ Live,6183,1666.7,100001.3,6000077,1666.7,15.44,0.75,6501,4399,2102,4077,21.70,77.24,45.6,23.7,27.4,3.4,0.3,1.8,38.3,59.6,64537,74197,1976125.6,1372.3,26.63,0.55,357312.8,72623,837035
3,2021-08-18 08:00:00,2021-08-18 09:00:00,2021-08-18,08:00,08,Total,HQ Live,3672,1155.5,69331.9,4159911,1155.5,17.53,1.03,3962,2534,1428,2364,22.17,75.92,40.9,25.9,26.3,6.9,0.2,1.5,39.3,59.0,64537,74197,1976125.6,1372.3,26.63,0.55,357312.8,72623,837035
4,2021-08-18 11:00:00,2021-08-18 12:00:00,2021-08-18,11:00,11,Total,HQ Live,3490,1542.8,92570.9,5554251,1542.8,24.43,1.63,3796,1949,1847,1774,32.04,77.43,35.9,22.7,29.5,12.0,0.1,0.8,34.8,64.3,64537,74197,1976125.6,1372.3,26.63,0.55,357312.8,72623,837035


In [ ]:
df_query = df_bq.query("Hour == '11'")
df_query.head()

,window_start,window_end,Day,Time_Window,Hour,platform,stream,Unique_Devices,Hours_Total,Minutes_Total,Seconds_Total,AMA,Minutes_Avg,Minutes_Median,Sessions_R,New_sessions_R,Existing_sessions_R,Session_Ends,New_Segment_Completes,Existing_Segment_Completes,Bounce_Percentage_1,Active_Percentage_1,Engaged_Percentage_1,Passive_Percentage_1,Bounce_Percentage_2,Active_Percentage_2,Engaged_Percentage_2,Passive_Percentage_2,Unique_Devices_Day,Sessions_Day,Minutes_Day,AMA_Day,Minutes_Avg_Day,Minutes_Median_Day,ad_minutes,streams,ad_starts
4,2021-08-18 11:00:00,2021-08-18 12:00:00,2021-08-18,11:00,11,Total,HQ Live,3490,1542.8,92570.9,5554251,1542.8,24.43,1.63,3796,1949,1847,1774,32.04,77.43,35.9,22.7,29.5,12.0,0.1,0.8,34.8,64.3,64537,74197,1976125.6,1372.3,26.63,0.55,357312.8,72623,837035
22,2021-08-18 23:00:00,2021-08-19 00:00:00,2021-08-18,23:00,11,Total,HQ Live,4769,1307.3,78439.7,4706380,1307.3,15.43,1.33,5107,2889,2218,3406,25.85,67.45,33.8,32.9,27.8,5.5,0.2,2.0,43.7,54.1,64537,74197,1976125.6,1372.3,26.63,0.55,357312.8,72623,837035
35,2021-08-18 11:00:00,2021-08-18 12:00:00,2021-08-18,11:00,11,Total,Other Live,73,44.4,2665.0,159899,44.4,36.51,0.44,73,26,47,27,23.08,85.11,50.0,19.2,23.1,7.7,0.1,0.4,8.1,91.5,4260,4590,195544.5,135.8,42.60,5.17,2444.1,5261,9454
39,2021-08-18 23:00:00,2021-08-19 00:00:00,2021-08-18,23:00,11,Total,Other Live,64,31.8,1907.4,114442,31.8,29.80,0.27,64,21,43,36,9.52,61.36,61.9,19.0,14.3,4.8,0.1,0.4,28.0,71.6,4260,4590,195544.5,135.8,42.60,5.17,2444.1,5261,9454
56,2021-08-19 11:00:00,2021-08-19 12:00:00,2021-08-19,11:00,11,Total,HQ Live,3671,1520.0,91197.7,5471860,1520.0,23.16,1.45,3944,2187,1757,2005,26.94,77.88,33.5,28.2,28.6,9.6,0.1,1.1,31.0,67.7,78685,89852,2130609.3,1479.6,23.71,0.57,341744.3,96117,837420


In [ ]:
#Sort
df_sorted = df_bq.sort_values(by='AMA', ascending=True)
df_sorted.head()

,window_start,window_end,Day,Time_Window,Hour,platform,stream,Unique_Devices,Hours_Total,Minutes_Total,Seconds_Total,AMA,Minutes_Avg,Minutes_Median,Sessions_R,New_sessions_R,Existing_sessions_R,Session_Ends,New_Segment_Completes,Existing_Segment_Completes,Bounce_Percentage_1,Active_Percentage_1,Engaged_Percentage_1,Passive_Percentage_1,Bounce_Percentage_2,Active_Percentage_2,Engaged_Percentage_2,Passive_Percentage_2,Unique_Devices_Day,Sessions_Day,Minutes_Day,AMA_Day,Minutes_Avg_Day,Minutes_Median_Day,ad_minutes,streams,ad_starts
21594,2022-11-11 02:00:00,2022-11-11 03:00:00,2022-11-11,02:00,02,Total,Other Live,5,0.0,2.2,131,0.0,0.44,0.20,5,5,0,5,0.00,NaN,60.0,40.0,0.0,0.0,12.8,87.2,NaN,NaN,9656,10121,292259.2,203.0,28.88,15.57,1946.6,16419,4646
14078,2022-06-07 04:00:00,2022-06-07 05:00:00,2022-06-07,04:00,04,Total,Other Live,2,0.0,0.5,31,0.0,0.26,0.52,2,1,1,1,0.00,100.0,0.0,100.0,0.0,0.0,0.0,100.0,NaN,NaN,4015,5009,243936.8,169.4,48.70,20.52,1719.1,6319,4766
14083,2022-06-07 03:00:00,2022-06-07 04:00:00,2022-06-07,03:00,03,Total,Other Live,3,0.0,0.3,19,0.0,0.11,0.26,3,2,1,2,0.00,100.0,100.0,0.0,0.0,0.0,100.0,NaN,NaN,NaN,4015,5009,243936.8,169.4,48.70,20.52,1719.1,6319,4766
14649,2022-06-19 06:00:00,2022-06-19 07:00:00,2022-06-19,06:00,06,Total,Other Live,9,0.0,0.1,4,0.0,0.01,0.82,10,9,1,9,11.11,0.0,44.4,33.3,22.2,0.0,100.0,0.0,NaN,NaN,3387,3833,166023.6,115.3,43.31,18.35,1268.9,6937,3502
12448,2022-05-04 04:00:00,2022-05-04 05:00:00,2022-05-04,04:00,04,Total,Other Live,5,0.0,0.2,12,0.0,0.04,0.00,5,5,0,5,0.00,NaN,100.0,0.0,0.0,0.0,100.0,NaN,NaN,NaN,22614,25468,1538710.7,1068.5,60.42,40.58,64466.1,52769,171217


In [ ]:
df_sorted = df_bq.sort_values(by=['AMA', 'Hours_Total'])
df_sorted.head()

,window_start,window_end,Day,Time_Window,Hour,platform,stream,Unique_Devices,Hours_Total,Minutes_Total,Seconds_Total,AMA,Minutes_Avg,Minutes_Median,Sessions_R,New_sessions_R,Existing_sessions_R,Session_Ends,New_Segment_Completes,Existing_Segment_Completes,Bounce_Percentage_1,Active_Percentage_1,Engaged_Percentage_1,Passive_Percentage_1,Bounce_Percentage_2,Active_Percentage_2,Engaged_Percentage_2,Passive_Percentage_2,Unique_Devices_Day,Sessions_Day,Minutes_Day,AMA_Day,Minutes_Avg_Day,Minutes_Median_Day,ad_minutes,streams,ad_starts
12440,2022-05-04 03:00:00,2022-05-04 04:00:00,2022-05-04,03:00,03,Total,Other Live,3,0.0,2.0,120,0.0,0.67,0.00,3,3,0,3,0.00,NaN,66.7,33.3,0.0,0.0,0.0,100.0,NaN,NaN,22614,25468,1538710.7,1068.5,60.42,40.58,64466.1,52769,171217
12448,2022-05-04 04:00:00,2022-05-04 05:00:00,2022-05-04,04:00,04,Total,Other Live,5,0.0,0.2,12,0.0,0.04,0.00,5,5,0,5,0.00,NaN,100.0,0.0,0.0,0.0,100.0,NaN,NaN,NaN,22614,25468,1538710.7,1068.5,60.42,40.58,64466.1,52769,171217
14078,2022-06-07 04:00:00,2022-06-07 05:00:00,2022-06-07,04:00,04,Total,Other Live,2,0.0,0.5,31,0.0,0.26,0.52,2,1,1,1,0.00,100.0,0.0,100.0,0.0,0.0,0.0,100.0,NaN,NaN,4015,5009,243936.8,169.4,48.70,20.52,1719.1,6319,4766
14083,2022-06-07 03:00:00,2022-06-07 04:00:00,2022-06-07,03:00,03,Total,Other Live,3,0.0,0.3,19,0.0,0.11,0.26,3,2,1,2,0.00,100.0,100.0,0.0,0.0,0.0,100.0,NaN,NaN,NaN,4015,5009,243936.8,169.4,48.70,20.52,1719.1,6319,4766
14649,2022-06-19 06:00:00,2022-06-19 07:00:00,2022-06-19,06:00,06,Total,Other Live,9,0.0,0.1,4,0.0,0.01,0.82,10,9,1,9,11.11,0.0,44.4,33.3,22.2,0.0,100.0,0.0,NaN,NaN,3387,3833,166023.6,115.3,43.31,18.35,1268.9,6937,3502


In [ ]:
#Sample
#Pandas’ .sample() function allows you to randomly select rows from a DataFrame. It returns a new DataFrame containing the randomly selected rows. The function takes several parameters that allow you to control the sampling process, such as the number of rows to return, and whether or not to sample with replacement and seed for reproducibility.

# Sample 2 rows without replacement
df_sample = df_bq.sample(n=2, replace=False, random_state=1)
df_sample

,window_start,window_end,Day,Time_Window,Hour,platform,stream,Unique_Devices,Hours_Total,Minutes_Total,Seconds_Total,AMA,Minutes_Avg,Minutes_Median,Sessions_R,New_sessions_R,Existing_sessions_R,Session_Ends,New_Segment_Completes,Existing_Segment_Completes,Bounce_Percentage_1,Active_Percentage_1,Engaged_Percentage_1,Passive_Percentage_1,Bounce_Percentage_2,Active_Percentage_2,Engaged_Percentage_2,Passive_Percentage_2,Unique_Devices_Day,Sessions_Day,Minutes_Day,AMA_Day,Minutes_Avg_Day,Minutes_Median_Day,ad_minutes,streams,ad_starts
23394,2022-12-19 21:00:00,2022-12-19 22:00:00,2022-12-19,21:00,09,Total,Other Live,64,9.5,570.1,34204,9.5,8.91,0.17,64,59,5,50,18.64,NaN,64.4,10.2,25.4,0.0,0.7,3.5,74.8,21.1,1608,1703,182507.3,126.7,107.17,9.87,255.4,2083,694
25765,2023-02-06 07:00:00,2023-02-06 08:00:00,2023-02-06,07:00,07,Total,HQ Live,6132,1001.6,60096.4,3605784,1001.6,9.63,1.50,6249,4956,1293,4799,17.39,NaN,24.5,45.8,27.4,2.2,0.5,6.5,54.4,38.6,113294,131976,1950060.3,1354.2,14.78,0.82,358741.4,206502,966967


In [ ]:
# Sample 2 rows without replacement with specific column to be chosen
df_sample = df_bq.sample(n=2, replace=False, random_state=1, axis=1)
df_sample

,Day,Sessions_Day
0,2021-08-18,74197
1,2021-08-18,74197
2,2021-08-18,74197
3,2021-08-18,74197
4,2021-08-18,74197
...,...,...
26443,2023-02-20,1563
26444,2023-02-20,94165
26445,2023-02-20,1563
26446,2023-02-20,94165


In [ ]:
#isnull
df_bq.isnull()

,window_start,window_end,Day,Time_Window,Hour,platform,stream,Unique_Devices,Hours_Total,Minutes_Total,Seconds_Total,AMA,Minutes_Avg,Minutes_Median,Sessions_R,New_sessions_R,Existing_sessions_R,Session_Ends,New_Segment_Completes,Existing_Segment_Completes,Bounce_Percentage_1,Active_Percentage_1,Engaged_Percentage_1,Passive_Percentage_1,Bounce_Percentage_2,Active_Percentage_2,Engaged_Percentage_2,Passive_Percentage_2,Unique_Devices_Day,Sessions_Day,Minutes_Day,AMA_Day,Minutes_Avg_Day,Minutes_Median_Day,ad_minutes,streams,ad_starts
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26443,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
26444,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
26445,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False
26446,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [ ]:
#fillna
df_bq.fillna(0)

# fill missing values in place. This will not return a new df but modify the original df
df_bq.fillna(0, inplace=True)

In [ ]:
#dropna()
#df.dropna() is a method used in the Pandas library to remove missing or null values from a DataFrame. It removes rows or columns from the DataFrame where at least one element is missing.
df = df_bq.dropna()

#If you want to remove only the columns that contain at least one missing value you can use df.dropna(axis=1)
df = df_bq.dropna(axis=1)

# You can also set thresh parameter to keep only the rows/columns that have at least thresh non-NA/null values
df = df_bq.dropna(thresh=2)


In [ ]:
#drop()
#This would drop the first row of the df
df_drop = df_bq.drop(0)

#This would remove the first and second rows of the DataFrame.
df_drop = df_bq.drop([0,1])

#Similarly, you can drop columns by passing the labels of the columns you want to remove and setting the axis parameter to 1
df_drop = df_bq.drop(['AMA', 'Hour'], axis=1)


In [ ]:
#pivot table
pivot_table = pd.pivot_table(df_bq, index='Hour', values='AMA', aggfunc='sum')
pivot_table.head()

,AMA
Hour,
01,4187343.1
02,4546226.2
03,5537033.3
04,6660229.3
05,6406526.7


In [ ]:
#Multiple indices
pivot_table = pd.pivot_table(df_bq, index=['Hour','platform'], values= 'AMA' , aggfunc=['sum','count'])
pivot_table.head(20)

,,sum,count
,,AMA,AMA
Hour,platform,,
01,Total,4187343.1,2204
02,Total,4546226.2,2204
03,Total,5537033.3,2204
04,Total,6660229.3,2204
05,Total,6406526.7,2204
06,Total,6237695.8,2204
07,Total,4994203.8,2204
08,Total,3605030.4,2204


In [ ]:
#Group By
grouped = df_bq.groupby('Hour')
# Print the mean of each group
grouped.mean()

,Unique_Devices,Hours_Total,Minutes_Total,Seconds_Total,AMA,Minutes_Avg,Minutes_Median,Sessions_R,New_sessions_R,Existing_sessions_R,Session_Ends,New_Segment_Completes,Existing_Segment_Completes,Bounce_Percentage_1,Active_Percentage_1,Engaged_Percentage_1,Passive_Percentage_1,Bounce_Percentage_2,Active_Percentage_2,Engaged_Percentage_2,Passive_Percentage_2,Unique_Devices_Day,Sessions_Day,Minutes_Day,AMA_Day,Minutes_Avg_Day,Minutes_Median_Day,ad_minutes,streams,ad_starts
Hour,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
01,8041.553539,1899.883439,113993.012114,6839580.649274,1899.883439,22.515336,5.912604,8266.42196,5929.462341,2336.959619,5547.713702,21.584446,62.265805,40.809710,24.778630,31.126633,3.284936,0.217794,1.764190,48.145289,50.164575,122112.088022,144033.075318,3.010583e+06,2090.682123,32.320172,8.981143,328052.365064,205903.919238,855476.547187
02,7406.258621,2062.716062,123762.928630,7425775.558984,2062.716062,23.423652,8.794609,7741.438748,4949.973684,2791.465064,4744.761343,23.560123,64.030207,40.786966,23.291462,32.635377,3.287511,0.208330,1.710697,45.556946,53.347203,122112.088022,144033.075318,3.010583e+06,2090.682123,32.320172,8.981143,328052.365064,205903.919238,855476.547187
03,9399.389746,2512.265563,150735.921007,9044155.185118,2512.265563,24.402173,9.198564,9797.472777,6736.896098,3060.576679,5772.988203,24.082625,66.173541,41.003920,22.684412,32.748496,3.565451,0.254763,1.822086,44.456944,54.861686,122112.088022,144033.075318,3.010583e+06,2090.682123,32.320172,8.981143,328052.365064,205903.919238,855476.547187
04,11094.060799,3021.882623,181313.001906,10878780.04265,3021.882623,23.940299,9.824158,11419.817604,7310.533122,4109.284483,7415.840744,21.115514,64.121282,41.387443,23.388399,31.857507,3.368926,0.329820,2.101138,44.841274,54.937928,122112.088022,144033.075318,3.010583e+06,2090.682123,32.320172,8.981143,328052.365064,205903.919238,855476.547187
05,9155.396552,2906.772550,174406.294918,10464377.535844,2906.772550,20.779705,12.051489,9595.20735,5501.84392,4093.36343,5762.32804,29.402674,64.777509,37.132864,22.255924,35.927054,4.684748,0.228663,1.799809,43.896066,54.668335,122112.088022,144033.075318,3.010583e+06,2090.682123,32.320172,8.981143,328052.365064,205903.919238,855476.547187
06,9639.151089,2830.170508,169810.229310,10188613.654719,2830.170508,18.521261,14.461348,10060.297641,6148.448276,3911.849365,6396.902904,30.272949,61.655186,33.880808,21.744555,40.802269,3.573730,0.315070,2.131416,54.096170,43.796093,122112.088022,144033.075318,3.010583e+06,2090.682123,32.320172,8.981143,328052.365064,205903.919238,855476.547187
07,9718.440109,2265.972686,135958.442105,8157506.394283,2265.972686,18.041171,14.562500,9940.98049,6192.013158,3748.967332,7577.749546,30.504941,58.392822,33.555717,22.161933,41.238067,3.045690,0.311842,2.415070,59.152091,38.315143,122112.088022,144033.075318,3.010583e+06,2090.682123,32.320172,8.981143,328052.365064,205903.919238,855476.547187
08,8191.670145,1635.676225,98140.508530,5888430.38657,1635.676225,18.045127,12.417790,8363.770871,5905.591652,2458.17922,6281.486842,28.570127,57.531641,35.956670,22.515290,38.744646,2.782849,0.335254,2.608352,61.138720,35.992458,122112.088022,144033.075318,3.010583e+06,2090.682123,32.320172,8.981143,328052.365064,205903.919238,855476.547187
09,7435.555808,1557.713113,93462.776044,5607766.413793,1557.713113,19.265717,6.542627,7606.918784,5437.06216,2169.856624,5558.848004,24.952877,57.772398,37.796370,23.438022,35.760254,3.004855,0.320009,2.496824,61.138294,36.055742,122112.088022,144033.075318,3.010583e+06,2090.682123,32.320172,8.981143,328052.365064,205903.919238,855476.547187


In [ ]:
grouped = df_bq.groupby(['Hour', 'platform'])

# Print the sum of each group
grouped.sum()

,,Unique_Devices,Hours_Total,Minutes_Total,Seconds_Total,AMA,Minutes_Avg,Minutes_Median,Sessions_R,New_sessions_R,Existing_sessions_R,Session_Ends,New_Segment_Completes,Existing_Segment_Completes,Bounce_Percentage_1,Active_Percentage_1,Engaged_Percentage_1,Passive_Percentage_1,Bounce_Percentage_2,Active_Percentage_2,Engaged_Percentage_2,Passive_Percentage_2,Unique_Devices_Day,Sessions_Day,Minutes_Day,AMA_Day,Minutes_Avg_Day,Minutes_Median_Day,ad_minutes,streams,ad_starts
Hour,platform,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
01,Total,17723584,4187343.1,251240598.7,15074435751,4187343.1,49623.80,13031.38,18219194,13068535,5150659,12227161,47572.12,120235.27,89944.6,54612.1,68603.1,7240.0,479.8,3823.0,105775.2,110311.9,269135042,317448898,6.635324e+09,4607863.4,71233.66,19794.44,723027412.6,453812238,1885470310
02,Total,16323394,4546226.2,272773494.7,16366409332,4546226.2,51625.73,19365.73,17062131,10909742,6152389,10457454,51879.39,123578.30,89812.9,51287.8,71863.1,7239.1,457.7,3582.2,99040.8,117310.5,269135042,317448898,6.635324e+09,4607863.4,71233.66,19794.44,723027412.6,453812238,1885470310
03,Total,20716255,5537033.3,332221969.9,19933318028,5537033.3,53782.39,20181.65,21593630,14848119,6745511,12723666,52837.28,127648.76,89962.6,49769.6,71850.2,7822.6,553.6,3704.3,95715.8,120421.4,269135042,317448898,6.635324e+09,4607863.4,71233.66,19794.44,723027412.6,453812238,1885470310
04,Total,24451310,6660229.3,399613856.2,23976831214,6660229.3,52764.42,21593.50,25169278,16112415,9056863,16344513,46411.90,123561.71,90969.6,51407.7,70022.8,7404.9,713.4,4246.4,95063.5,120369.0,269135042,317448898,6.635324e+09,4607863.4,71233.66,19794.44,723027412.6,453812238,1885470310
05,Total,20178494,6406526.7,384391474.0,23063488089,6406526.7,45798.47,26549.43,21147837,12126064,9021773,12700171,64774.09,124567.15,81803.7,49029.8,79147.3,10320.5,501.0,3770.6,95956.8,120161.0,269135042,317448898,6.635324e+09,4607863.4,71233.66,19794.44,723027412.6,453812238,1885470310
06,Total,21244689,6237695.8,374261745.4,22455704495,6237695.8,40820.86,31872.81,22172896,13551180,8621716,14098774,66721.58,119117.82,74673.3,47925.0,89928.2,7876.5,694.1,4667.8,118632.9,96395.2,269135042,317448898,6.635324e+09,4607863.4,71233.66,19794.44,723027412.6,453812238,1885470310
07,Total,21419442,4994203.8,299652406.4,17979144093,4994203.8,39762.74,32095.75,21909921,13647197,8262724,16701360,67232.89,112756.54,73956.8,48844.9,90888.7,6712.7,687.3,5320.4,130134.6,84255.0,269135042,317448898,6.635324e+09,4607863.4,71233.66,19794.44,723027412.6,453812238,1885470310
08,Total,18054441,3605030.4,216301680.8,12978100572,3605030.4,39771.46,27368.81,18433751,13015924,5417827,13844397,62968.56,111151.13,79248.5,49623.7,85393.2,6133.4,738.9,5746.2,134688.6,79219.4,269135042,317448898,6.635324e+09,4607863.4,71233.66,19794.44,723027412.6,453812238,1885470310
09,Total,16387965,3433199.7,205991958.4,12359517176,3433199.7,42461.64,14419.95,16765649,11983285,4782364,12251701,54996.14,111558.50,83303.2,51657.4,78815.6,6622.7,705.3,5503.0,134748.8,79430.8,269135042,317448898,6.635324e+09,4607863.4,71233.66,19794.44,723027412.6,453812238,1885470310


In [ ]:
#Transpose
df_transposed = df_bq.transpose()

# Print the transposed DataFrame
df_transposed.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,...,26398,26399,26400,26401,26402,26403,26404,26405,26406,26407,26408,26409,26410,26411,26412,26413,26414,26415,26416,26417,26418,26419,26420,26421,26422,26423,26424,26425,26426,26427,26428,26429,26430,26431,26432,26433,26434,26435,26436,26437,26438,26439,26440,26441,26442,26443,26444,26445,26446,26447
window_start,2021-08-18 09:00:00,2021-08-18 14:00:00,2021-08-18 18:00:00,2021-08-18 08:00:00,2021-08-18 11:00:00,2021-08-18 21:00:00,2021-08-18 20:00:00,2021-08-18 22:00:00,2021-08-18 17:00:00,2021-08-18 16:00:00,2021-08-18 06:00:00,2021-08-18 12:00:00,2021-08-18 01:00:00,2021-08-18 10:00:00,2021-08-18 13:00:00,2021-08-18 03:00:00,2021-08-18 19:00:00,2021-08-18 05:00:00,2021-08-18 15:00:00,2021-08-18 07:00:00,2021-08-18 04:00:00,2021-08-18 02:00:00,2021-08-18 23:00:00,2021-08-18 00:00:00,2021-08-18 05:00:00,2021-08-18 12:00:00,2021-08-18 17:00:00,2021-08-18 06:00:00,2021-08-18 22:00:00,2021-08-18 07:00:00,2021-08-18 18:00:00,2021-08-18 04:00:00,2021-08-18 03:00:00,2021-08-18 19:00:00,2021-08-18 16:00:00,2021-08-18 11:00:00,2021-08-18 09:00:00,2021-08-18 00:00:00,2021-08-18 15:00:00,2021-08-18 23:00:00,2021-08-18 08:00:00,2021-08-18 10:00:00,2021-08-18 14:00:00,2021-08-18 20:00:00,2021-08-18 13:00:00,2021-08-18 02:00:00,2021-08-18 21:00:00,2021-08-18 01:00:00,2021-08-19 15:00:00,2021-08-19 08:00:00,...,2023-02-19 23:00:00,2023-02-19 23:00:00,2023-02-20 00:00:00,2023-02-20 00:00:00,2023-02-20 01:00:00,2023-02-20 01:00:00,2023-02-20 02:00:00,2023-02-20 02:00:00,2023-02-20 03:00:00,2023-02-20 03:00:00,2023-02-20 04:00:00,2023-02-20 04:00:00,2023-02-20 05:00:00,2023-02-20 05:00:00,2023-02-20 06:00:00,2023-02-20 06:00:00,2023-02-20 07:00:00,2023-02-20 07:00:00,2023-02-20 08:00:00,2023-02-20 08:00:00,2023-02-20 09:00:00,2023-02-20 09:00:00,2023-02-20 10:00:00,2023-02-20 10:00:00,2023-02-20 11:00:00,2023-02-20 11:00:00,2023-02-20 12:00:00,2023-02-20 12:00:00,2023-02-20 13:00:00,2023-02-20 13:00:00,2023-02-20 14:00:00,2023-02-20 14:00:00,2023-02-20 15:00:00,2023-02-20 15:00:00,2023-02-20 16:00:00,2023-02-20 16:00:00,2023-02-20 17:00:00,2023-02-20 17:00:00,2023-02-20 18:00:00,2023-02-20 18:00:00,2023-02-20 19:00:00,2023-02-20 19:00:00,2023-02-20 20:00:00,2023-02-20 20:00:00,2023-02-20 21:00:00,2023-02-20 21:00:00,2023-02-20 22:00:00,2023-02-20 22:00:00,2023-02-20 23:00:00,2023-02-20 23:00:00
window_end,2021-08-18 10:00:00,2021-08-18 15:00:00,2021-08-18 19:00:00,2021-08-18 09:00:00,2021-08-18 12:00:00,2021-08-18 22:00:00,2021-08-18 21:00:00,2021-08-18 23:00:00,2021-08-18 18:00:00,2021-08-18 17:00:00,2021-08-18 07:00:00,2021-08-18 13:00:00,2021-08-18 02:00:00,2021-08-18 11:00:00,2021-08-18 14:00:00,2021-08-18 04:00:00,2021-08-18 20:00:00,2021-08-18 06:00:00,2021-08-18 16:00:00,2021-08-18 08:00:00,2021-08-18 05:00:00,2021-08-18 03:00:00,2021-08-19 00:00:00,2021-08-18 01:00:00,2021-08-18 06:00:00,2021-08-18 13:00:00,2021-08-18 18:00:00,2021-08-18 07:00:00,2021-08-18 23:00:00,2021-08-18 08:00:00,2021-08-18 19:00:00,2021-08-18 05:00:00,2021-08-18 04:00:00,2021-08-18 20:00:00,2021-08-18 17:00:00,2021-08-18 12:00:00,2021-08-18 10:00:00,2021-08-18 01:00:00,2021-08-18 16:00:00,2021-08-19 00:00:00,2021-08-18 09:00:00,2021-08-18 11:00:00,2021-08-18 15:00:00,2021-08-18 21:00:00,2021-08-18 14:00:00,2021-08-18 03:00:00,2021-08-18 22:00:00,2021-08-18 02:00:00,2021-08-19 16:00:00,2021-08-19 09:00:00,...,2023-02-20 00:00:00,2023-02-20 00:00:00,2023-02-20 01:00:00,2023-02-20 01:00:00,2023-02-20 02:00:00,2023-02-20 02:00:00,2023-02-20 03:00:00,2023-02-20 03:00:00,2023-02-20 04:00:00,2023-02-20 04:00:00,2023-02-20 05:00:00,2023-02-20 05:00:00,2023-02-20 06:00:00,2023-02-20 06:00:00,2023-02-20 07:00:00,2023-02-20 07:00:00,2023-02-20 08:00:00,2023-02-20 08:00:00,2023-02-20 09:00:00,2023-02-20 09:00:00,2023-02-20 10:00:00,2023-02-20 10:00:00,2023-02-20 11:00:00,2023-02-20 11:00:00,2023-02-20 12:00:00,2023-02-20 12:00:00,202

In [ ]:
#Another Transpose method

df_transposed = df_bq.T
df_transposed.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,...,26398,26399,26400,26401,26402,26403,26404,26405,26406,26407,26408,26409,26410,26411,26412,26413,26414,26415,26416,26417,26418,26419,26420,26421,26422,26423,26424,26425,26426,26427,26428,26429,26430,26431,26432,26433,26434,26435,26436,26437,26438,26439,26440,26441,26442,26443,26444,26445,26446,26447
window_start,2021-08-18 09:00:00,2021-08-18 14:00:00,2021-08-18 18:00:00,2021-08-18 08:00:00,2021-08-18 11:00:00,2021-08-18 21:00:00,2021-08-18 20:00:00,2021-08-18 22:00:00,2021-08-18 17:00:00,2021-08-18 16:00:00,2021-08-18 06:00:00,2021-08-18 12:00:00,2021-08-18 01:00:00,2021-08-18 10:00:00,2021-08-18 13:00:00,2021-08-18 03:00:00,2021-08-18 19:00:00,2021-08-18 05:00:00,2021-08-18 15:00:00,2021-08-18 07:00:00,2021-08-18 04:00:00,2021-08-18 02:00:00,2021-08-18 23:00:00,2021-08-18 00:00:00,2021-08-18 05:00:00,2021-08-18 12:00:00,2021-08-18 17:00:00,2021-08-18 06:00:00,2021-08-18 22:00:00,2021-08-18 07:00:00,2021-08-18 18:00:00,2021-08-18 04:00:00,2021-08-18 03:00:00,2021-08-18 19:00:00,2021-08-18 16:00:00,2021-08-18 11:00:00,2021-08-18 09:00:00,2021-08-18 00:00:00,2021-08-18 15:00:00,2021-08-18 23:00:00,2021-08-18 08:00:00,2021-08-18 10:00:00,2021-08-18 14:00:00,2021-08-18 20:00:00,2021-08-18 13:00:00,2021-08-18 02:00:00,2021-08-18 21:00:00,2021-08-18 01:00:00,2021-08-19 15:00:00,2021-08-19 08:00:00,...,2023-02-19 23:00:00,2023-02-19 23:00:00,2023-02-20 00:00:00,2023-02-20 00:00:00,2023-02-20 01:00:00,2023-02-20 01:00:00,2023-02-20 02:00:00,2023-02-20 02:00:00,2023-02-20 03:00:00,2023-02-20 03:00:00,2023-02-20 04:00:00,2023-02-20 04:00:00,2023-02-20 05:00:00,2023-02-20 05:00:00,2023-02-20 06:00:00,2023-02-20 06:00:00,2023-02-20 07:00:00,2023-02-20 07:00:00,2023-02-20 08:00:00,2023-02-20 08:00:00,2023-02-20 09:00:00,2023-02-20 09:00:00,2023-02-20 10:00:00,2023-02-20 10:00:00,2023-02-20 11:00:00,2023-02-20 11:00:00,2023-02-20 12:00:00,2023-02-20 12:00:00,2023-02-20 13:00:00,2023-02-20 13:00:00,2023-02-20 14:00:00,2023-02-20 14:00:00,2023-02-20 15:00:00,2023-02-20 15:00:00,2023-02-20 16:00:00,2023-02-20 16:00:00,2023-02-20 17:00:00,2023-02-20 17:00:00,2023-02-20 18:00:00,2023-02-20 18:00:00,2023-02-20 19:00:00,2023-02-20 19:00:00,2023-02-20 20:00:00,2023-02-20 20:00:00,2023-02-20 21:00:00,2023-02-20 21:00:00,2023-02-20 22:00:00,2023-02-20 22:00:00,2023-02-20 23:00:00,2023-02-20 23:00:00
window_end,2021-08-18 10:00:00,2021-08-18 15:00:00,2021-08-18 19:00:00,2021-08-18 09:00:00,2021-08-18 12:00:00,2021-08-18 22:00:00,2021-08-18 21:00:00,2021-08-18 23:00:00,2021-08-18 18:00:00,2021-08-18 17:00:00,2021-08-18 07:00:00,2021-08-18 13:00:00,2021-08-18 02:00:00,2021-08-18 11:00:00,2021-08-18 14:00:00,2021-08-18 04:00:00,2021-08-18 20:00:00,2021-08-18 06:00:00,2021-08-18 16:00:00,2021-08-18 08:00:00,2021-08-18 05:00:00,2021-08-18 03:00:00,2021-08-19 00:00:00,2021-08-18 01:00:00,2021-08-18 06:00:00,2021-08-18 13:00:00,2021-08-18 18:00:00,2021-08-18 07:00:00,2021-08-18 23:00:00,2021-08-18 08:00:00,2021-08-18 19:00:00,2021-08-18 05:00:00,2021-08-18 04:00:00,2021-08-18 20:00:00,2021-08-18 17:00:00,2021-08-18 12:00:00,2021-08-18 10:00:00,2021-08-18 01:00:00,2021-08-18 16:00:00,2021-08-19 00:00:00,2021-08-18 09:00:00,2021-08-18 11:00:00,2021-08-18 15:00:00,2021-08-18 21:00:00,2021-08-18 14:00:00,2021-08-18 03:00:00,2021-08-18 22:00:00,2021-08-18 02:00:00,2021-08-19 16:00:00,2021-08-19 09:00:00,...,2023-02-20 00:00:00,2023-02-20 00:00:00,2023-02-20 01:00:00,2023-02-20 01:00:00,2023-02-20 02:00:00,2023-02-20 02:00:00,2023-02-20 03:00:00,2023-02-20 03:00:00,2023-02-20 04:00:00,2023-02-20 04:00:00,2023-02-20 05:00:00,2023-02-20 05:00:00,2023-02-20 06:00:00,2023-02-20 06:00:00,2023-02-20 07:00:00,2023-02-20 07:00:00,2023-02-20 08:00:00,2023-02-20 08:00:00,2023-02-20 09:00:00,2023-02-20 09:00:00,2023-02-20 10:00:00,2023-02-20 10:00:00,2023-02-20 11:00:00,2023-02-20 11:00:00,2023-02-20 12:00:00,2023-02-20 12:00:00,202

In [ ]:
# Merge()

# Create the first DataFrame
df1 = pd.DataFrame({'key': ['A', 'B', 'C', 'D'],
                   'value': [1, 2, 3, 4]})

# Create the second DataFrame
df2 = pd.DataFrame({'key': ['B', 'D', 'E', 'F'],
                   'value': [5, 6, 7, 8]})

# Merge the two DataFrames on the 'key' column
merged_df = df1.merge(df2, on='key')

# Print the merged DataFrame
merged_df

,key,value_x,value_y
0,B,2,5
1,D,4,6


In [ ]:
#You can also use left, right and outer join by passing how = ‘left’, how = ‘right’, or how = ‘outer’ respectively

#You can also merge multiple columns by passing a list of columns to the on parameter
df1 = pd.DataFrame({'key1': ['A', 'B', 'C', 'D'],
                    'key2': ['A', 'B', 'C', 'D'],
                   'value': [1, 2, 3, 4]})

# Create the second DataFrame
df2 = pd.DataFrame({'key1': ['B', 'D', 'E', 'F'],
                    'key2': ['B', 'D', 'E', 'F'],
                   'value': [5, 6, 7, 8]})
merged_df = df1.merge(df2, on=['key1','key2'])
merged_df

,key1,key2,value_x,value_y
0,B,B,2,5
1,D,D,4,6


In [ ]:
# You can also specify a different column name to merge by using left_on and right_on parameters

merged_df = df1.merge(df2, left_on='key1', right_on='key3')

In [ ]:
# rename()

# df.rename() is a pandas function that allows you to change the name of one or more columns or rows in a DataFrame. You can use the columns parameter to change the column names, and the index parameter to change the row names

# Rename column 'Count' to 'count'
df_rename = df_bq.rename(columns={'AMA': 'ama'})
df_rename.head()

,window_start,window_end,Day,Time_Window,Hour,platform,stream,Unique_Devices,Hours_Total,Minutes_Total,Seconds_Total,ama,Minutes_Avg,Minutes_Median,Sessions_R,New_sessions_R,Existing_sessions_R,Session_Ends,New_Segment_Completes,Existing_Segment_Completes,Bounce_Percentage_1,Active_Percentage_1,Engaged_Percentage_1,Passive_Percentage_1,Bounce_Percentage_2,Active_Percentage_2,Engaged_Percentage_2,Passive_Percentage_2,Unique_Devices_Day,Sessions_Day,Minutes_Day,AMA_Day,Minutes_Avg_Day,Minutes_Median_Day,ad_minutes,streams,ad_starts
0,2021-08-18 09:00:00,2021-08-18 10:00:00,2021-08-18,09:00,09,Total,HQ Live,6219,1420.0,85201.4,5112082,1420.0,13.08,1.00,6520,4893,1627,4886,10.68,73.45,40.1,31.0,25.6,3.3,0.3,3.2,42.0,54.4,64537,74197,1976125.6,1372.3,26.63,0.55,357312.8,72623,837035
1,2021-08-18 14:00:00,2021-08-18 15:00:00,2021-08-18,14:00,02,Total,HQ Live,9572,1884.6,113073.1,6784384,1884.6,11.46,0.87,9878,7496,2382,7559,9.58,73.62,37.3,48.3,12.7,1.6,0.8,2.6,37.5,59.0,64537,74197,1976125.6,1372.3,26.63,0.55,357312.8,72623,837035
2,2021-08-18 18:00:00,2021-08-18 19:00:00,2021-08-18,18:00,06,Total,HQ Live,6183,1666.7,100001.3,6000077,1666.7,15.44,0.75,6501,4399,2102,4077,21.70,77.24,45.6,23.7,27.4,3.4,0.3,1.8,38.3,59.6,64537,74197,1976125.6,1372.3,26.63,0.55,357312.8,72623,837035
3,2021-08-18 08:00:00,2021-08-18 09:00:00,2021-08-18,08:00,08,Total,HQ Live,3672,1155.5,69331.9,4159911,1155.5,17.53,1.03,3962,2534,1428,2364,22.17,75.92,40.9,25.9,26.3,6.9,0.2,1.5,39.3,59.0,64537,74197,1976125.6,1372.3,26.63,0.55,357312.8,72623,837035
4,2021-08-18 11:00:00,2021-08-18 12:00:00,2021-08-18,11:00,11,Total,HQ Live,3490,1542.8,92570.9,5554251,1542.8,24.43,1.63,3796,1949,1847,1774,32.04,77.43,35.9,22.7,29.5,12.0,0.1,0.8,34.8,64.3,64537,74197,1976125.6,1372.3,26.63,0.55,357312.8,72623,837035


In [ ]:
# You can also use a dictionary to rename multiple columns at once

df_rename = df_bq.rename(columns={'AMA': 'ama', 'Hour':'hour'})
df_rename.head()

,window_start,window_end,Day,Time_Window,hour,platform,stream,Unique_Devices,Hours_Total,Minutes_Total,Seconds_Total,ama,Minutes_Avg,Minutes_Median,Sessions_R,New_sessions_R,Existing_sessions_R,Session_Ends,New_Segment_Completes,Existing_Segment_Completes,Bounce_Percentage_1,Active_Percentage_1,Engaged_Percentage_1,Passive_Percentage_1,Bounce_Percentage_2,Active_Percentage_2,Engaged_Percentage_2,Passive_Percentage_2,Unique_Devices_Day,Sessions_Day,Minutes_Day,AMA_Day,Minutes_Avg_Day,Minutes_Median_Day,ad_minutes,streams,ad_starts
0,2021-08-18 09:00:00,2021-08-18 10:00:00,2021-08-18,09:00,09,Total,HQ Live,6219,1420.0,85201.4,5112082,1420.0,13.08,1.00,6520,4893,1627,4886,10.68,73.45,40.1,31.0,25.6,3.3,0.3,3.2,42.0,54.4,64537,74197,1976125.6,1372.3,26.63,0.55,357312.8,72623,837035
1,2021-08-18 14:00:00,2021-08-18 15:00:00,2021-08-18,14:00,02,Total,HQ Live,9572,1884.6,113073.1,6784384,1884.6,11.46,0.87,9878,7496,2382,7559,9.58,73.62,37.3,48.3,12.7,1.6,0.8,2.6,37.5,59.0,64537,74197,1976125.6,1372.3,26.63,0.55,357312.8,72623,837035
2,2021-08-18 18:00:00,2021-08-18 19:00:00,2021-08-18,18:00,06,Total,HQ Live,6183,1666.7,100001.3,6000077,1666.7,15.44,0.75,6501,4399,2102,4077,21.70,77.24,45.6,23.7,27.4,3.4,0.3,1.8,38.3,59.6,64537,74197,1976125.6,1372.3,26.63,0.55,357312.8,72623,837035
3,2021-08-18 08:00:00,2021-08-18 09:00:00,2021-08-18,08:00,08,Total,HQ Live,3672,1155.5,69331.9,4159911,1155.5,17.53,1.03,3962,2534,1428,2364,22.17,75.92,40.9,25.9,26.3,6.9,0.2,1.5,39.3,59.0,64537,74197,1976125.6,1372.3,26.63,0.55,357312.8,72623,837035
4,2021-08-18 11:00:00,2021-08-18 12:00:00,2021-08-18,11:00,11,Total,HQ Live,3490,1542.8,92570.9,5554251,1542.8,24.43,1.63,3796,1949,1847,1774,32.04,77.43,35.9,22.7,29.5,12.0,0.1,0.8,34.8,64.3,64537,74197,1976125.6,1372.3,26.63,0.55,357312.8,72623,837035


In [ ]:
df_rename = df_bq.rename(index={0:'first',1:'second',2:'third'})
df_rename.head()

,window_start,window_end,Day,Time_Window,Hour,platform,stream,Unique_Devices,Hours_Total,Minutes_Total,Seconds_Total,AMA,Minutes_Avg,Minutes_Median,Sessions_R,New_sessions_R,Existing_sessions_R,Session_Ends,New_Segment_Completes,Existing_Segment_Completes,Bounce_Percentage_1,Active_Percentage_1,Engaged_Percentage_1,Passive_Percentage_1,Bounce_Percentage_2,Active_Percentage_2,Engaged_Percentage_2,Passive_Percentage_2,Unique_Devices_Day,Sessions_Day,Minutes_Day,AMA_Day,Minutes_Avg_Day,Minutes_Median_Day,ad_minutes,streams,ad_starts
first,2021-08-18 09:00:00,2021-08-18 10:00:00,2021-08-18,09:00,09,Total,HQ Live,6219,1420.0,85201.4,5112082,1420.0,13.08,1.00,6520,4893,1627,4886,10.68,73.45,40.1,31.0,25.6,3.3,0.3,3.2,42.0,54.4,64537,74197,1976125.6,1372.3,26.63,0.55,357312.8,72623,837035
second,2021-08-18 14:00:00,2021-08-18 15:00:00,2021-08-18,14:00,02,Total,HQ Live,9572,1884.6,113073.1,6784384,1884.6,11.46,0.87,9878,7496,2382,7559,9.58,73.62,37.3,48.3,12.7,1.6,0.8,2.6,37.5,59.0,64537,74197,1976125.6,1372.3,26.63,0.55,357312.8,72623,837035
third,2021-08-18 18:00:00,2021-08-18 19:00:00,2021-08-18,18:00,06,Total,HQ Live,6183,1666.7,100001.3,6000077,1666.7,15.44,0.75,6501,4399,2102,4077,21.70,77.24,45.6,23.7,27.4,3.4,0.3,1.8,38.3,59.6,64537,74197,1976125.6,1372.3,26.63,0.55,357312.8,72623,837035
3,2021-08-18 08:00:00,2021-08-18 09:00:00,2021-08-18,08:00,08,Total,HQ Live,3672,1155.5,69331.9,4159911,1155.5,17.53,1.03,3962,2534,1428,2364,22.17,75.92,40.9,25.9,26.3,6.9,0.2,1.5,39.3,59.0,64537,74197,1976125.6,1372.3,26.63,0.55,357312.8,72623,837035
4,2021-08-18 11:00:00,2021-08-18 12:00:00,2021-08-18,11:00,11,Total,HQ Live,3490,1542.8,92570.9,5554251,1542.8,24.43,1.63,3796,1949,1847,1774,32.04,77.43,35.9,22.7,29.5,12.0,0.1,0.8,34.8,64.3,64537,74197,1976125.6,1372.3,26.63,0.55,357312.8,72623,837035


In [ ]:
#Unique Values
df_bq.AMA.unique()

array([1420. , 1884.6, 1666.7, ..., 1357.3,  859.4,  351.4])

In [ ]:
#Total unique values in a column
df_bq.Hour.nunique()

12

In [ ]:
#Correlation
df_bq.corr(method='pearson')

,Unique_Devices,Hours_Total,Minutes_Total,Seconds_Total,AMA,Minutes_Avg,Minutes_Median,Sessions_R,New_sessions_R,Existing_sessions_R,Session_Ends,New_Segment_Completes,Existing_Segment_Completes,Bounce_Percentage_1,Active_Percentage_1,Engaged_Percentage_1,Passive_Percentage_1,Bounce_Percentage_2,Active_Percentage_2,Engaged_Percentage_2,Passive_Percentage_2,Unique_Devices_Day,Sessions_Day,Minutes_Day,AMA_Day,Minutes_Avg_Day,Minutes_Median_Day,ad_minutes,streams,ad_starts
Unique_Devices,1.000000,0.802330,0.802330,0.802330,0.802330,-0.215230,0.002083,0.998717,0.936478,0.852474,0.937739,-0.016211,-0.121206,-0.099301,0.148120,0.028668,-0.079047,0.066608,0.283853,0.220076,-0.256756,0.547825,0.549029,0.459117,0.459117,-0.139776,-0.082206,0.490049,0.551787,0.479234
Hours_Total,0.802330,1.000000,1.000000,1.000000,1.000000,0.016163,0.064500,0.793588,0.588079,0.913621,0.590626,0.118187,-0.033923,-0.176129,-0.005207,0.174326,-0.042119,-0.003907,0.015479,0.193569,-0.194053,0.308288,0.307487,0.470797,0.470797,0.046685,0.083397,0.303040,0.340911,0.298448
Minutes_Total,0.802330,1.000000,1.000000,1.000000,1.000000,0.016163,0.064500,0.793588,0.588079,0.913621,0.590626,0.118187,-0.033923,-0.176129,-0.005207,0.174326,-0.042119,-0.003907,0.015480,0.193569,-0.194053,0.308288,0.307487,0.470797,0.470797,0.046685,0.083397,0.303040,0.340911,0.298448
Seconds_Total,0.802330,1.000000,1.000000,1.000000,1.000000,0.016163,0.064500,0.793588,0.588079,0.913621,0.590626,0.118187,-0.033923,-0.176129,-0.005207,0.174326,-0.042119,-0.003907,0.015480,0.193569,-0.194053,0.308288,0.307487,0.470797,0.470797,0.046685,0.083397,0.303040,0.340911,0.298448
AMA,0.802330,1.000000,1.000000,1.000000,1.000000,0.016163,0.064500,0.793588,0.588079,0.913621,0.590626,0.118187,-0.033923,-0.176129,-0.005207,0.174326,-0.042119,-0.003907,0.015479,0.193569,-0.194053,0.308288,0.307487,0.470797,0.470797,0.046685,0.083397,0.303040,0.340911,0.298448
Minutes_Avg,-0.215230,0.016163,0.016163,0.016163,0.016163,1.000000,0.186208,-0.218141,-0.294618,-0.049845,-0.293284,0.310303,0.446979,-0.114094,-0.459999,0.298113,0.242102,-0.195431,-0.579751,-0.313654,0.389262,-0.285559,-0.278208,0.030148,0.030148,0.609370,0.457149,-0.243121,-0.222491,-0.223575
Minutes_Median,0.002083,0.064500,0.064500,0.064500,0.064500,0.186208,1.000000,0.000051,-0.014335,0.021816,-0.043415,0.447499,0.097168,-0.321997,-0.253007,0.336396,0.394536,-0.053132,-0.146847,0.154899,-0.101644,-0.106532,-0.103631,-0.001741,-0.001741,0.189730,0.215194,-0.104022,-0.083711,-0.096524
Sessions_R,0.998717,0.793588,0.793588,0.793588,0.793588,-0.218141,0.000051,1.000000,0.938453,0.852401,0.938883,-0.018890,-0.114578,-0.095859,0.149782,0.024518,-0.078691,0.066702,0.285298,0.216609,-0.253605,0.556247,0.557760,0.458225,0.458225,-0.144327,-0.086674,0.497135,0.559375,0.485799
New_sessions_R,0.936478,0.588079,0.588079,0.588079,0.588079,-0.294618,-0.014335,0.938453,1.000000,0.619328,0.923980,-0.056113,-0.117289,-0.054492,0.188731,-0.035227,-0.076850,0.094930,0.382971,0.200008,-0.250354,0.574736,0.576060,0.376727,0.376727,-0.211133,-0.150760,0.497757,0.562017,0.485151
Existing_sessions_R,0.852474,0.913621,0.913621,0.913621,0.913621,-0.049845,0.021816,0.852401,0.619328,1.000000,0.735387,0.042006,-0.084699,-0.135398,0.054755,0.109055,-0.062531,0.007909,0.068715,0.189533,-0.197450,0.394329,0.395763,0.471272,0.471272,-0.008444,0.031210,0.376498,0.420694,0.369815


In [ ]:
#info
df_bq.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26448 entries, 0 to 26447
Data columns (total 37 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   window_start                26448 non-null  datetime64[ns]
 1   window_end                  26448 non-null  datetime64[ns]
 2   Day                         26448 non-null  dbdate        
 3   Time_Window                 26448 non-null  object        
 4   Hour                        26448 non-null  object        
 5   platform                    26448 non-null  object        
 6   stream                      26448 non-null  object        
 7   Unique_Devices              26448 non-null  Int64         
 8   Hours_Total                 26448 non-null  float64       
 9   Minutes_Total               26448 non-null  float64       
 10  Seconds_Total               26448 non-null  Int64         
 11  AMA                         26448 non-null  float64   

In [ ]:
#Find N largest/smallest values
df_bq.nlargest(6, 'AMA')
df_bq.nsmallest(6, 'AMA')

,window_start,window_end,Day,Time_Window,Hour,platform,stream,Unique_Devices,Hours_Total,Minutes_Total,Seconds_Total,AMA,Minutes_Avg,Minutes_Median,Sessions_R,New_sessions_R,Existing_sessions_R,Session_Ends,New_Segment_Completes,Existing_Segment_Completes,Bounce_Percentage_1,Active_Percentage_1,Engaged_Percentage_1,Passive_Percentage_1,Bounce_Percentage_2,Active_Percentage_2,Engaged_Percentage_2,Passive_Percentage_2,Unique_Devices_Day,Sessions_Day,Minutes_Day,AMA_Day,Minutes_Avg_Day,Minutes_Median_Day,ad_minutes,streams,ad_starts
12440,2022-05-04 03:00:00,2022-05-04 04:00:00,2022-05-04,03:00,03,Total,Other Live,3,0.0,2.0,120,0.0,0.67,0.00,3,3,0,3,0.00,NaN,66.7,33.3,0.0,0.0,0.0,100.0,NaN,NaN,22614,25468,1538710.7,1068.5,60.42,40.58,64466.1,52769,171217
12448,2022-05-04 04:00:00,2022-05-04 05:00:00,2022-05-04,04:00,04,Total,Other Live,5,0.0,0.2,12,0.0,0.04,0.00,5,5,0,5,0.00,NaN,100.0,0.0,0.0,0.0,100.0,NaN,NaN,NaN,22614,25468,1538710.7,1068.5,60.42,40.58,64466.1,52769,171217
14078,2022-06-07 04:00:00,2022-06-07 05:00:00,2022-06-07,04:00,04,Total,Other Live,2,0.0,0.5,31,0.0,0.26,0.52,2,1,1,1,0.00,100.0,0.0,100.0,0.0,0.0,0.0,100.0,NaN,NaN,4015,5009,243936.8,169.4,48.70,20.52,1719.1,6319,4766
14083,2022-06-07 03:00:00,2022-06-07 04:00:00,2022-06-07,03:00,03,Total,Other Live,3,0.0,0.3,19,0.0,0.11,0.26,3,2,1,2,0.00,100.0,100.0,0.0,0.0,0.0,100.0,NaN,NaN,NaN,4015,5009,243936.8,169.4,48.70,20.52,1719.1,6319,4766
14649,2022-06-19 06:00:00,2022-06-19 07:00:00,2022-06-19,06:00,06,Total,Other Live,9,0.0,0.1,4,0.0,0.01,0.82,10,9,1,9,11.11,0.0,44.4,33.3,22.2,0.0,100.0,0.0,NaN,NaN,3387,3833,166023.6,115.3,43.31,18.35,1268.9,6937,3502
14867,2022-06-23 04:00:00,2022-06-23 05:00:00,2022-06-23,04:00,04,Total,Other Live,1,0.0,0.6,36,0.0,0.60,0.60,1,1,0,1,0.00,NaN,0.0,100.0,0.0,0.0,NaN,100.0,NaN,NaN,1872,2390,106331.6,73.8,44.49,12.43,6.2,2583,17


In [ ]:
#Binning
#we can create bins to divide the data into a certain range. The Binning method is used to normalize the data and make it smooth by removing noise from the data
pd.cut(df_bq['Unique_Devices'], bins = 500).value_counts()

(-411.545, 826.09]        10222
(2476.27, 3301.36]         1639
(826.09, 1651.18]          1474
(1651.18, 2476.27]         1292
(3301.36, 4126.45]         1258
                          ...  
(272280.7, 273105.79]         0
(273105.79, 273930.88]        0
(273930.88, 274755.97]        0
(274755.97, 275581.06]        0
(206273.5, 207098.59]         0
Name: Unique_Devices, Length: 500, dtype: int64

In [ ]:
#Concat

df1 = pd.DataFrame([['a', 1], ['b', 2]],
                   columns=['letter', 'number'])
df2 = pd.DataFrame([['c', 3], ['d', 4]],
                   columns=['letter', 'number'])

pd.concat([df1, df2])

,letter,number
0,a,1
1,b,2
0,c,3
1,d,4


In [ ]:
#Across Columns
df1 = pd.DataFrame([['a', 1], 
                    ['b', 2]],
                   columns=['letter', 'number'])
df2 = pd.DataFrame([[1, 'c'], 
                    [2, 'd']],
                   columns=['letter', 'number'])

pd.concat([df1, df2], axis=0)

,letter,number
0,a,1
1,b,2
0,1,c
1,2,d


In [ ]:
#Across Rows
df1 = pd.DataFrame([['a', 1], 
                    ['b', 2]],
                   columns=['letter', 'number'])
df2 = pd.DataFrame([[1, 'c'], 
                    [2, 'd']],
                   columns=['letter', 'number'])

pd.concat([df1, df2], axis=1
          )

,letter,number,letter,number
0,a,1,1,c
1,b,2,2,d


In [ ]:
#Drop Duplicates
df_bq.drop_duplicates()

,window_start,window_end,Day,Time_Window,Hour,platform,stream,Unique_Devices,Hours_Total,Minutes_Total,Seconds_Total,AMA,Minutes_Avg,Minutes_Median,Sessions_R,New_sessions_R,Existing_sessions_R,Session_Ends,New_Segment_Completes,Existing_Segment_Completes,Bounce_Percentage_1,Active_Percentage_1,Engaged_Percentage_1,Passive_Percentage_1,Bounce_Percentage_2,Active_Percentage_2,Engaged_Percentage_2,Passive_Percentage_2,Unique_Devices_Day,Sessions_Day,Minutes_Day,AMA_Day,Minutes_Avg_Day,Minutes_Median_Day,ad_minutes,streams,ad_starts
0,2021-08-18 09:00:00,2021-08-18 10:00:00,2021-08-18,09:00,09,Total,HQ Live,6219,1420.0,85201.4,5112082,1420.0,13.08,1.00,6520,4893,1627,4886,10.68,73.45,40.1,31.0,25.6,3.3,0.3,3.2,42.0,54.4,64537,74197,1976125.6,1372.3,26.63,0.55,357312.8,72623,837035
1,2021-08-18 14:00:00,2021-08-18 15:00:00,2021-08-18,14:00,02,Total,HQ Live,9572,1884.6,113073.1,6784384,1884.6,11.46,0.87,9878,7496,2382,7559,9.58,73.62,37.3,48.3,12.7,1.6,0.8,2.6,37.5,59.0,64537,74197,1976125.6,1372.3,26.63,0.55,357312.8,72623,837035
2,2021-08-18 18:00:00,2021-08-18 19:00:00,2021-08-18,18:00,06,Total,HQ Live,6183,1666.7,100001.3,6000077,1666.7,15.44,0.75,6501,4399,2102,4077,21.70,77.24,45.6,23.7,27.4,3.4,0.3,1.8,38.3,59.6,64537,74197,1976125.6,1372.3,26.63,0.55,357312.8,72623,837035
3,2021-08-18 08:00:00,2021-08-18 09:00:00,2021-08-18,08:00,08,Total,HQ Live,3672,1155.5,69331.9,4159911,1155.5,17.53,1.03,3962,2534,1428,2364,22.17,75.92,40.9,25.9,26.3,6.9,0.2,1.5,39.3,59.0,64537,74197,1976125.6,1372.3,26.63,0.55,357312.8,72623,837035
4,2021-08-18 11:00:00,2021-08-18 12:00:00,2021-08-18,11:00,11,Total,HQ Live,3490,1542.8,92570.9,5554251,1542.8,24.43,1.63,3796,1949,1847,1774,32.04,77.43,35.9,22.7,29.5,12.0,0.1,0.8,34.8,64.3,64537,74197,1976125.6,1372.3,26.63,0.55,357312.8,72623,837035
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26443,2023-02-20 21:00:00,2023-02-20 22:00:00,2023-02-20,21:00,09,Total,Other Live,84,23.3,1399.1,83945,23.3,16.66,0.48,84,52,32,72,13.46,NaN,50.0,15.4,34.6,0.0,0.1,1.3,95.9,2.6,1424,1563,66684.1,46.3,42.66,8.17,796.7,2045,2180
26444,2023-02-20 22:00:00,2023-02-20 23:00:00,2023-02-20,22:00,10,Total,HQ Live,6730,859.4,51561.6,3093694,859.4,7.60,1.48,6810,4397,2413,5016,19.42,NaN,27.5,37.6,34.9,0.0,0.6,6.4,80.8,12.2,79894,94165,1490011.4,1034.7,15.82,0.78,324856.7,146193,860620
26445,2023-02-20 22:00:00,2023-02-20 23:00:00,2023-02-20,22:00,10,Total,Other Live,35,8.7,522.2,31330,8.7,14.92,0.38,35,23,12,29,8.70,NaN,65.2,8.7,26.1,0.0,0.5,0.7,98.8,NaN,1424,1563,66684.1,46.3,42.66,8.17,796.7,2045,2180
26446,2023-02-20 23:00:00,2023-02-21 00:00:00,2023-02-20,23:00,11,Total,HQ Live,5120,351.4,21083.1,1264984,351.4,4.10,1.10,5136,3246,1890,5218,0.00,NaN,32.9,40.9,26.1,0.0,1.3,11.0,80.0,7.7,79894,94165,1490011.4,1034.7,15.82,0.78,324856.7,146193,860620


In [ ]:
#Value Counts
df_bq.AMA.value_counts()

4.0       25
4.1       24
5.4       24
9.5       23
3.0       23
          ..
2935.1     1
2300.8     1
1633.0     1
262.6      1
351.4      1
Name: AMA, Length: 15499, dtype: int64

In [ ]:
#Insert
#When we want to add a new column to a dataframe, it is added at the end by default. However, pandas offers the option to add the new column in any position using insert function

new_col = df_bq['AMA']
#insert the new column at position 2
df_bq.insert(2, 'AMA_Duplicate', new_col)
df_bq

,window_start,window_end,AMA_Duplicate,Day,Time_Window,Hour,platform,stream,Unique_Devices,Hours_Total,Minutes_Total,Seconds_Total,AMA,Minutes_Avg,Minutes_Median,Sessions_R,New_sessions_R,Existing_sessions_R,Session_Ends,New_Segment_Completes,Existing_Segment_Completes,Bounce_Percentage_1,Active_Percentage_1,Engaged_Percentage_1,Passive_Percentage_1,Bounce_Percentage_2,Active_Percentage_2,Engaged_Percentage_2,Passive_Percentage_2,Unique_Devices_Day,Sessions_Day,Minutes_Day,AMA_Day,Minutes_Avg_Day,Minutes_Median_Day,ad_minutes,streams,ad_starts
0,2021-08-18 09:00:00,2021-08-18 10:00:00,1420.0,2021-08-18,09:00,09,Total,HQ Live,6219,1420.0,85201.4,5112082,1420.0,13.08,1.00,6520,4893,1627,4886,10.68,73.45,40.1,31.0,25.6,3.3,0.3,3.2,42.0,54.4,64537,74197,1976125.6,1372.3,26.63,0.55,357312.8,72623,837035
1,2021-08-18 14:00:00,2021-08-18 15:00:00,1884.6,2021-08-18,14:00,02,Total,HQ Live,9572,1884.6,113073.1,6784384,1884.6,11.46,0.87,9878,7496,2382,7559,9.58,73.62,37.3,48.3,12.7,1.6,0.8,2.6,37.5,59.0,64537,74197,1976125.6,1372.3,26.63,0.55,357312.8,72623,837035
2,2021-08-18 18:00:00,2021-08-18 19:00:00,1666.7,2021-08-18,18:00,06,Total,HQ Live,6183,1666.7,100001.3,6000077,1666.7,15.44,0.75,6501,4399,2102,4077,21.70,77.24,45.6,23.7,27.4,3.4,0.3,1.8,38.3,59.6,64537,74197,1976125.6,1372.3,26.63,0.55,357312.8,72623,837035
3,2021-08-18 08:00:00,2021-08-18 09:00:00,1155.5,2021-08-18,08:00,08,Total,HQ Live,3672,1155.5,69331.9,4159911,1155.5,17.53,1.03,3962,2534,1428,2364,22.17,75.92,40.9,25.9,26.3,6.9,0.2,1.5,39.3,59.0,64537,74197,1976125.6,1372.3,26.63,0.55,357312.8,72623,837035
4,2021-08-18 11:00:00,2021-08-18 12:00:00,1542.8,2021-08-18,11:00,11,Total,HQ Live,3490,1542.8,92570.9,5554251,1542.8,24.43,1.63,3796,1949,1847,1774,32.04,77.43,35.9,22.7,29.5,12.0,0.1,0.8,34.8,64.3,64537,74197,1976125.6,1372.3,26.63,0.55,357312.8,72623,837035
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26443,2023-02-20 21:00:00,2023-02-20 22:00:00,23.3,2023-02-20,21:00,09,Total,Other Live,84,23.3,1399.1,83945,23.3,16.66,0.48,84,52,32,72,13.46,NaN,50.0,15.4,34.6,0.0,0.1,1.3,95.9,2.6,1424,1563,66684.1,46.3,42.66,8.17,796.7,2045,2180
26444,2023-02-20 22:00:00,2023-02-20 23:00:00,859.4,2023-02-20,22:00,10,Total,HQ Live,6730,859.4,51561.6,3093694,859.4,7.60,1.48,6810,4397,2413,5016,19.42,NaN,27.5,37.6,34.9,0.0,0.6,6.4,80.8,12.2,79894,94165,1490011.4,1034.7,15.82,0.78,324856.7,146193,860620
26445,2023-02-20 22:00:00,2023-02-20 23:00:00,8.7,2023-02-20,22:00,10,Total,Other Live,35,8.7,522.2,31330,8.7,14.92,0.38,35,23,12,29,8.70,NaN,65.2,8.7,26.1,0.0,0.5,0.7,98.8,NaN,1424,1563,66684.1,46.3,42.66,8.17,796.7,2045,2180
26446,2023-02-20 23:00:00,2023-02-21 00:00:00,351.4,2023-02-20,23:00,11,Total,HQ Live,5120,351.4,21083.1,1264984,351.4,4.10,1.10,5136,3246,1890,5218,0.00,NaN,32.9,40.9,26.1,0.0,1.3,11.0,80.0,7.7,79894,94165,1490011.4,1034.7,15.82,0.78,324856.7,146193,860620


In [ ]:
#Where
# “Where” is used to replace values in rows or columns based on a condition. The default replacement values is NaN but we can also specify the value to be put as a replacement
df_bq['AMA'].where(df_bq['AMA'] > 500 , 0)

0        1420.0
1        1884.6
2        1666.7
3        1155.5
4        1542.8
          ...  
26443       0.0
26444     859.4
26445       0.0
26446       0.0
26447       0.0
Name: AMA, Length: 26448, dtype: float64

In [ ]:
#isin
hours = ['06']
df_bq[df_bq.Hour.isin(hours)]

,window_start,window_end,AMA_Duplicate,Day,Time_Window,Hour,platform,stream,Unique_Devices,Hours_Total,Minutes_Total,Seconds_Total,AMA,Minutes_Avg,Minutes_Median,Sessions_R,New_sessions_R,Existing_sessions_R,Session_Ends,New_Segment_Completes,Existing_Segment_Completes,Bounce_Percentage_1,Active_Percentage_1,Engaged_Percentage_1,Passive_Percentage_1,Bounce_Percentage_2,Active_Percentage_2,Engaged_Percentage_2,Passive_Percentage_2,Unique_Devices_Day,Sessions_Day,Minutes_Day,AMA_Day,Minutes_Avg_Day,Minutes_Median_Day,ad_minutes,streams,ad_starts
2,2021-08-18 18:00:00,2021-08-18 19:00:00,1666.7,2021-08-18,18:00,06,Total,HQ Live,6183,1666.7,100001.3,6000077,1666.7,15.44,0.75,6501,4399,2102,4077,21.70,77.24,45.6,23.7,27.4,3.4,0.3,1.8,38.3,59.6,64537,74197,1976125.6,1372.3,26.63,0.55,357312.8,72623,837035
10,2021-08-18 06:00:00,2021-08-18 07:00:00,878.1,2021-08-18,06:00,06,Total,HQ Live,2366,878.1,52687.8,3161270,878.1,20.00,1.35,2637,1489,1148,1390,24.36,80.46,36.8,24.6,33.0,5.5,0.1,1.0,32.4,66.4,64537,74197,1976125.6,1372.3,26.63,0.55,357312.8,72623,837035
27,2021-08-18 06:00:00,2021-08-18 07:00:00,76.6,2021-08-18,06:00,06,Total,Other Live,134,76.6,4594.4,275667,76.6,34.29,66.68,134,86,48,36,66.28,85.42,8.1,2.3,75.6,14.0,0.0,0.1,38.4,61.4,4260,4590,195544.5,135.8,42.60,5.17,2444.1,5261,9454
30,2021-08-18 18:00:00,2021-08-18 19:00:00,77.2,2021-08-18,18:00,06,Total,Other Live,214,77.2,4631.9,277911,77.2,21.34,41.22,218,143,75,79,58.04,74.67,22.4,14.0,63.6,0.0,0.1,0.8,60.8,38.3,4260,4590,195544.5,135.8,42.60,5.17,2444.1,5261,9454
50,2021-08-19 18:00:00,2021-08-19 19:00:00,1813.2,2021-08-19,18:00,06,Total,HQ Live,7430,1813.2,108794.1,6527646,1813.2,14.09,0.80,7726,5562,2164,5249,17.97,75.09,41.9,29.4,26.2,2.4,0.3,2.3,39.4,58.0,78685,89852,2130609.3,1479.6,23.71,0.57,341744.3,96117,837420
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26389,2023-02-19 18:00:00,2023-02-19 19:00:00,13872.3,2023-02-19,18:00,06,Total,Other Live,41134,13872.3,832337.6,49940254,13872.3,20.23,5.56,41153,6612,34541,41114,0.73,NaN,15.7,32.4,51.9,0.0,0.0,0.6,98.1,1.3,115408,124845,5725420.5,3976.0,45.86,18.32,391841.1,172767,1239447
26412,2023-02-20 06:00:00,2023-02-20 07:00:00,715.9,2023-02-20,06:00,06,Total,HQ Live,3118,715.9,42955.6,2577336,715.9,13.50,1.60,3182,2330,852,2252,20.73,NaN,28.9,36.6,31.8,2.7,0.3,3.7,52.2,43.8,79894,94165,1490011.4,1034.7,15.82,0.78,324856.7,146193,860620
26413,2023-02-20 06:00:00,2023-02-20 07:00:00,56.4,2023-02-20,06:00,06,Total,Other Live,128,56.4,3385.4,203126,56.4,26.45,44.07,128,111,17,49,57.66,NaN,15.3,8.1,70.3,6.3,0.0,0.5,74.5,24.9,1424,1563,66684.1,46.3,42.66,8.17,796.7,2045,2180
26436,2023-02-20 18:00:00,2023-02-20 19:00:00,1310.2,2023-02-20,18:00,06,Total,HQ Live,7019,1310.2,78612.4,4716746,1310.2,11.11,1.70,7098,5193,1905,4982,23.19,NaN,30.8,30.4,36.0,2.8,0.4,4.0,63.1,32.5,79894,94165,1490011.4,1034.7,15.82,0.78,324856.7,146193,860620


In [ ]:
#Rank
df_bq['rank_1'] = df_bq['AMA'].rank()
df_bq

,window_start,window_end,AMA_Duplicate,Day,Time_Window,Hour,platform,stream,Unique_Devices,Hours_Total,Minutes_Total,Seconds_Total,AMA,Minutes_Avg,Minutes_Median,Sessions_R,New_sessions_R,Existing_sessions_R,Session_Ends,New_Segment_Completes,Existing_Segment_Completes,Bounce_Percentage_1,Active_Percentage_1,Engaged_Percentage_1,Passive_Percentage_1,Bounce_Percentage_2,Active_Percentage_2,Engaged_Percentage_2,Passive_Percentage_2,Unique_Devices_Day,Sessions_Day,Minutes_Day,AMA_Day,Minutes_Avg_Day,Minutes_Median_Day,ad_minutes,streams,ad_starts,rank_1
0,2021-08-18 09:00:00,2021-08-18 10:00:00,1420.0,2021-08-18,09:00,09,Total,HQ Live,6219,1420.0,85201.4,5112082,1420.0,13.08,1.00,6520,4893,1627,4886,10.68,73.45,40.1,31.0,25.6,3.3,0.3,3.2,42.0,54.4,64537,74197,1976125.6,1372.3,26.63,0.55,357312.8,72623,837035,18344.0
1,2021-08-18 14:00:00,2021-08-18 15:00:00,1884.6,2021-08-18,14:00,02,Total,HQ Live,9572,1884.6,113073.1,6784384,1884.6,11.46,0.87,9878,7496,2382,7559,9.58,73.62,37.3,48.3,12.7,1.6,0.8,2.6,37.5,59.0,64537,74197,1976125.6,1372.3,26.63,0.55,357312.8,72623,837035,20560.0
2,2021-08-18 18:00:00,2021-08-18 19:00:00,1666.7,2021-08-18,18:00,06,Total,HQ Live,6183,1666.7,100001.3,6000077,1666.7,15.44,0.75,6501,4399,2102,4077,21.70,77.24,45.6,23.7,27.4,3.4,0.3,1.8,38.3,59.6,64537,74197,1976125.6,1372.3,26.63,0.55,357312.8,72623,837035,19601.0
3,2021-08-18 08:00:00,2021-08-18 09:00:00,1155.5,2021-08-18,08:00,08,Total,HQ Live,3672,1155.5,69331.9,4159911,1155.5,17.53,1.03,3962,2534,1428,2364,22.17,75.92,40.9,25.9,26.3,6.9,0.2,1.5,39.3,59.0,64537,74197,1976125.6,1372.3,26.63,0.55,357312.8,72623,837035,16440.0
4,2021-08-18 11:00:00,2021-08-18 12:00:00,1542.8,2021-08-18,11:00,11,Total,HQ Live,3490,1542.8,92570.9,5554251,1542.8,24.43,1.63,3796,1949,1847,1774,32.04,77.43,35.9,22.7,29.5,12.0,0.1,0.8,34.8,64.3,64537,74197,1976125.6,1372.3,26.63,0.55,357312.8,72623,837035,19004.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26443,2023-02-20 21:00:00,2023-02-20 22:00:00,23.3,2023-02-20,21:00,09,Total,Other Live,84,23.3,1399.1,83945,23.3,16.66,0.48,84,52,32,72,13.46,NaN,50.0,15.4,34.6,0.0,0.1,1.3,95.9,2.6,1424,1563,66684.1,46.3,42.66,8.17,796.7,2045,2180,2796.5
26444,2023-02-20 22:00:00,2023-02-20 23:00:00,859.4,2023-02-20,22:00,10,Total,HQ Live,6730,859.4,51561.6,3093694,859.4,7.60,1.48,6810,4397,2413,5016,19.42,NaN,27.5,37.6,34.9,0.0,0.6,6.4,80.8,12.2,79894,94165,1490011.4,1034.7,15.82,0.78,324856.7,146193,860620,13934.0
26445,2023-02-20 22:00:00,2023-02-20 23:00:00,8.7,2023-02-20,22:00,10,Total,Other Live,35,8.7,522.2,31330,8.7,14.92,0.38,35,23,12,29,8.70,NaN,65.2,8.7,26.1,0.0,0.5,0.7,98.8,NaN,1424,1563,66684.1,46.3,42.66,8.17,796.7,2045,2180,1136.5
26446,2023-02-20 23:00:00,2023-02-21 00:00:00,351.4,2023-02-20,23:00,11,Total,HQ Live,5120,351.4,21083.1,1264984,351.4,4.10,1.10,5136,3246,1890,5218,0.00,NaN,32.9,40.9,26.1,0.0,1.3,11.0,80.0,7.7,79894,94165,1490011.4,1034.7,15.82,0.78,324856.7,146193,860620,10227.0


In [ ]:
#Melt
# Melt is used to convert wide dataframes to narrow ones

df_bq.melt(id_vars=['Hour'])


,Hour,variable,value
0,09,window_start,2021-08-18 09:00:00
1,02,window_start,2021-08-18 14:00:00
2,06,window_start,2021-08-18 18:00:00
3,08,window_start,2021-08-18 08:00:00
4,11,window_start,2021-08-18 11:00:00
...,...,...,...
1005019,09,rank_1,2796.5
1005020,10,rank_1,13934.0
1005021,10,rank_1,1136.5
1005022,11,rank_1,10227.0


In [ ]:
#Replace a value. Replaces 09 in hour by 9
df_bq.replace('09','9')
#Can also pass a dictionary
df_bq.replace({'09':'9', '06':'6'})

,window_start,window_end,AMA_Duplicate,Day,Time_Window,Hour,platform,stream,Unique_Devices,Hours_Total,Minutes_Total,Seconds_Total,AMA,Minutes_Avg,Minutes_Median,Sessions_R,New_sessions_R,Existing_sessions_R,Session_Ends,New_Segment_Completes,Existing_Segment_Completes,Bounce_Percentage_1,Active_Percentage_1,Engaged_Percentage_1,Passive_Percentage_1,Bounce_Percentage_2,Active_Percentage_2,Engaged_Percentage_2,Passive_Percentage_2,Unique_Devices_Day,Sessions_Day,Minutes_Day,AMA_Day,Minutes_Avg_Day,Minutes_Median_Day,ad_minutes,streams,ad_starts,rank_1
0,2021-08-18 09:00:00,2021-08-18 10:00:00,1420.0,2021-08-18,09:00,9,Total,HQ Live,6219,1420.0,85201.4,5112082,1420.0,13.08,1.00,6520,4893,1627,4886,10.68,73.45,40.1,31.0,25.6,3.3,0.3,3.2,42.0,54.4,64537,74197,1976125.6,1372.3,26.63,0.55,357312.8,72623,837035,18344.0
1,2021-08-18 14:00:00,2021-08-18 15:00:00,1884.6,2021-08-18,14:00,02,Total,HQ Live,9572,1884.6,113073.1,6784384,1884.6,11.46,0.87,9878,7496,2382,7559,9.58,73.62,37.3,48.3,12.7,1.6,0.8,2.6,37.5,59.0,64537,74197,1976125.6,1372.3,26.63,0.55,357312.8,72623,837035,20560.0
2,2021-08-18 18:00:00,2021-08-18 19:00:00,1666.7,2021-08-18,18:00,06,Total,HQ Live,6183,1666.7,100001.3,6000077,1666.7,15.44,0.75,6501,4399,2102,4077,21.70,77.24,45.6,23.7,27.4,3.4,0.3,1.8,38.3,59.6,64537,74197,1976125.6,1372.3,26.63,0.55,357312.8,72623,837035,19601.0
3,2021-08-18 08:00:00,2021-08-18 09:00:00,1155.5,2021-08-18,08:00,08,Total,HQ Live,3672,1155.5,69331.9,4159911,1155.5,17.53,1.03,3962,2534,1428,2364,22.17,75.92,40.9,25.9,26.3,6.9,0.2,1.5,39.3,59.0,64537,74197,1976125.6,1372.3,26.63,0.55,357312.8,72623,837035,16440.0
4,2021-08-18 11:00:00,2021-08-18 12:00:00,1542.8,2021-08-18,11:00,11,Total,HQ Live,3490,1542.8,92570.9,5554251,1542.8,24.43,1.63,3796,1949,1847,1774,32.04,77.43,35.9,22.7,29.5,12.0,0.1,0.8,34.8,64.3,64537,74197,1976125.6,1372.3,26.63,0.55,357312.8,72623,837035,19004.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26443,2023-02-20 21:00:00,2023-02-20 22:00:00,23.3,2023-02-20,21:00,9,Total,Other Live,84,23.3,1399.1,83945,23.3,16.66,0.48,84,52,32,72,13.46,NaN,50.0,15.4,34.6,0.0,0.1,1.3,95.9,2.6,1424,1563,66684.1,46.3,42.66,8.17,796.7,2045,2180,2796.5
26444,2023-02-20 22:00:00,2023-02-20 23:00:00,859.4,2023-02-20,22:00,10,Total,HQ Live,6730,859.4,51561.6,3093694,859.4,7.60,1.48,6810,4397,2413,5016,19.42,NaN,27.5,37.6,34.9,0.0,0.6,6.4,80.8,12.2,79894,94165,1490011.4,1034.7,15.82,0.78,324856.7,146193,860620,13934.0
26445,2023-02-20 22:00:00,2023-02-20 23:00:00,8.7,2023-02-20,22:00,10,Total,Other Live,35,8.7,522.2,31330,8.7,14.92,0.38,35,23,12,29,8.70,NaN,65.2,8.7,26.1,0.0,0.5,0.7,98.8,NaN,1424,1563,66684.1,46.3,42.66,8.17,796.7,2045,2180,1136.5
26446,2023-02-20 23:00:00,2023-02-21 00:00:00,351.4,2023-02-20,23:00,11,Total,HQ Live,5120,351.4,21083.1,1264984,351.4,4.10,1.10,5136,3246,1890,5218,0.00,NaN,32.9,40.9,26.1,0.0,1.3,11.0,80.0,7.7,79894,94165,1490011.4,1034.7,15.82,0.78,324856.7,146193,860620,10227.0


window_start                  datetime64[ns]
window_end                    datetime64[ns]
AMA_Duplicate                        float64
Day                                   dbdate
Time_Window                           object
Hour                                  object
platform                              object
stream                                object
Unique_Devices                         Int64
Hours_Total                          float64
Minutes_Total                        float64
Seconds_Total                          Int64
AMA                                  float64
Minutes_Avg                          float64
Minutes_Median                       float64
Sessions_R                             Int64
New_sessions_R                         Int64
Existing_sessions_R                    Int64
Session_Ends                           Int64
New_Segment_Completes                float64
Existing_Segment_Completes           float64
Bounce_Percentage_1                  float64
Active_Per